In [19]:
import pandas as pd
from scipy.interpolate import CubicSpline
import numpy as np

In [20]:
df = pd.read_csv("./data/gdp/OECD.SDD.NAD,DSD_NAMAIN1@DF_QNA_EXPENDITURE_GROWTH_OECD,1.0+Q..CAN+DEU+JPN+FRA+ITA+GBR+USA+IND+BRA+CHN...B1GQ......G1..csv")

In [21]:
df.columns

Index(['STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION', 'FREQ',
       'Frequency of observation', 'ADJUSTMENT', 'Adjustment', 'REF_AREA',
       'Reference area', 'SECTOR', 'Institutional sector',
       'COUNTERPART_SECTOR', 'Counterpart institutional sector', 'TRANSACTION',
       'Transaction', 'INSTR_ASSET',
       'Financial instruments and non-financial assets', 'ACTIVITY',
       'Economic activity', 'EXPENDITURE', 'Expenditure', 'UNIT_MEASURE',
       'Unit of measure', 'PRICE_BASE', 'Price base', 'TRANSFORMATION',
       'Transformation', 'TABLE_IDENTIFIER', 'Table identifier', 'TIME_PERIOD',
       'Time period', 'OBS_VALUE', 'Observation value', 'REF_YEAR_PRICE',
       'Price reference year', 'BASE_PER', 'Base period', 'CONF_STATUS',
       'Confidentiality status', 'DECIMALS', 'Decimals', 'OBS_STATUS',
       'Observation status', 'UNIT_MULT', 'Unit multiplier', 'CURRENCY',
       'Currency'],
      dtype='object')

In [22]:
df = df[['TIME_PERIOD', 'Reference area', 'OBS_VALUE']]

In [23]:
df

,TIME_PERIOD,Reference area,OBS_VALUE
0,2001-Q3,France,0.548936
1,2001-Q4,France,-0.490594
2,2002-Q1,France,0.616558
3,2002-Q2,France,0.519235
4,2002-Q3,France,0.376161
...,...,...,...
2131,1963-Q2,Italy,2.741399
2132,1960-Q2,Italy,1.467603
2133,1960-Q3,Italy,1.468635
2134,1960-Q4,Italy,0.076755


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2136 entries, 0 to 2135
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   TIME_PERIOD     2136 non-null   object 
 1   Reference area  2136 non-null   object 
 2   OBS_VALUE       2136 non-null   float64
dtypes: float64(1), object(2)
memory usage: 50.2+ KB


In [25]:
df.sort_values("TIME_PERIOD", inplace=True)

In [26]:
df

,TIME_PERIOD,Reference area,OBS_VALUE
1823,1947-Q2,United States,-0.265224
1824,1947-Q3,United States,-0.204879
1825,1947-Q4,United States,1.565987
1826,1948-Q1,United States,1.506038
1814,1948-Q2,United States,1.652377
...,...,...,...
546,2024-Q1,Brazil,0.776849
90,2024-Q1,France,0.156524
1845,2024-Q1,China (People’s Republic of),1.600000
1565,2024-Q1,United States,0.311258


In [27]:
data = df.copy()

In [28]:
# Step 1: Convert 'TIME_PERIOD' to datetime by mapping quarters to the first day of the quarter
data['TIME_PERIOD'] = data['TIME_PERIOD'].str.replace('Q1', '01').str.replace('Q2', '04').str.replace('Q3', '07').str.replace('Q4', '10')
data['TIME_PERIOD'] = pd.to_datetime(data['TIME_PERIOD'], format='%Y-%m')


# Step 2: Prepare an empty list to store the results
results = []

# Step 3: Group the data by country and calculate monthly growth
for country, group in data.groupby('Reference area'):
    # Sort the group by 'TIME_PERIOD'
    group = group.sort_values('TIME_PERIOD')
    
    # Iterate through each quarter
    for i, row in group.iterrows():
        start_date = row['TIME_PERIOD']
        end_date = start_date + pd.DateOffset(months=3) - pd.DateOffset(days=1)  # End of the quarter

        # Generate the monthly dates within the quarter
        monthly_dates = pd.date_range(start=start_date, end=end_date, freq='M')

        # Calculate the monthly growth rate such that the compounded growth equals the quarterly growth
        quarterly_growth = row['OBS_VALUE']
        monthly_growth = np.sign(quarterly_growth) * (np.abs(1 + quarterly_growth) ** (1/3) - 1)
        
        # Create a DataFrame for the interpolated monthly values
        interpolated_df = pd.DataFrame({
            'TIME_PERIOD': monthly_dates,
            'Reference area': country,
            'Interpolated OBS_VALUE': [monthly_growth] * len(monthly_dates)
        })
        
        # Append the interpolated data for the quarter
        results.append(interpolated_df)

# Step 4: Concatenate all results into a single DataFrame
final_data = pd.concat(results).reset_index(drop=True)

# Display the final DataFrame with monthly compounded growth
print(final_data)

/var/folders/sh/s_xv2d8x6kv1_qyc9qgmkv300000gn/T/ipykernel_81884/390288054.py:20: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_dates = pd.date_range(start=start_date, end=end_date, freq='M')
/var/folders/sh/s_xv2d8x6kv1_qyc9qgmkv300000gn/T/ipykernel_81884/390288054.py:20: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_dates = pd.date_range(start=start_date, end=end_date, freq='M')
/var/folders/sh/s_xv2d8x6kv1_qyc9qgmkv300000gn/T/ipykernel_81884/390288054.py:20: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_dates = pd.date_range(start=start_date, end=end_date, freq='M')
/var/folders/sh/s_xv2d8x6kv1_qyc9qgmkv300000gn/T/ipykernel_81884/390288054.py:20: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_dates = pd.date_range(start=start_date, end=

     TIME_PERIOD Reference area  Interpolated OBS_VALUE
0     1996-04-30         Brazil                0.226194
1     1996-05-31         Brazil                0.226194
2     1996-06-30         Brazil                0.226194
3     1996-07-31         Brazil                0.696571
4     1996-08-31         Brazil                0.696571
...          ...            ...                     ...
6403  2023-11-30  United States                0.225031
6404  2023-12-31  United States                0.225031
6405  2024-01-31  United States                0.094534
6406  2024-02-29  United States                0.094534
6407  2024-03-31  United States                0.094534

[6408 rows x 3 columns]


In [29]:
data

,TIME_PERIOD,Reference area,OBS_VALUE
1823,1947-04-01,United States,-0.265224
1824,1947-07-01,United States,-0.204879
1825,1947-10-01,United States,1.565987
1826,1948-01-01,United States,1.506038
1814,1948-04-01,United States,1.652377
...,...,...,...
546,2024-01-01,Brazil,0.776849
90,2024-01-01,France,0.156524
1845,2024-01-01,China (People’s Republic of),1.600000
1565,2024-01-01,United States,0.311258


In [30]:
final_data[final_data["Reference area"] == "United States"].tail(16)


,TIME_PERIOD,Reference area,Interpolated OBS_VALUE
6392,2022-12-31,United States,0.178175
6393,2023-01-31,United States,0.158880
6394,2023-02-28,United States,0.158880
6395,2023-03-31,United States,0.158880
6396,2023-04-30,United States,0.147537
6397,2023-05-31,United States,0.147537
6398,2023-06-30,United States,0.147537
6399,2023-07-31,United States,0.299383
6400,2023-08-31,United States,0.299383
6401,2023-09-30,United States,0.299383


In [31]:
data[data["Reference area"] == "United States"].tail(4)

,TIME_PERIOD,Reference area,OBS_VALUE
1562,2023-04-01,United States,0.511122
1563,2023-07-01,United States,1.193871
1564,2023-10-01,United States,0.838405
1565,2024-01-01,United States,0.311258


In [32]:
final_data = final_data.sort_values("TIME_PERIOD")
final_data

,TIME_PERIOD,Reference area,Interpolated OBS_VALUE
5484,1947-04-30,United States,0.097629
5485,1947-05-31,United States,0.097629
5486,1947-06-30,United States,0.097629
5487,1947-07-31,United States,0.073573
5488,1947-08-31,United States,0.073573
...,...,...,...
2018,2024-03-31,France,0.049667
2786,2024-03-31,Germany,0.066658
3887,2024-03-31,Italy,0.103363
335,2024-03-31,Brazil,0.211203


In [33]:
final_data["TIME_PERIOD"] = final_data['TIME_PERIOD'].dt.strftime('%Y-%m')
final_data

,TIME_PERIOD,Reference area,Interpolated OBS_VALUE
5484,1947-04,United States,0.097629
5485,1947-05,United States,0.097629
5486,1947-06,United States,0.097629
5487,1947-07,United States,0.073573
5488,1947-08,United States,0.073573
...,...,...,...
2018,2024-03,France,0.049667
2786,2024-03,Germany,0.066658
3887,2024-03,Italy,0.103363
335,2024-03,Brazil,0.211203


In [34]:
final_data.rename(columns = {"Reference area": "Country", "Interpolated OBS_VALUE" : "gdp_growth_MoM"}, inplace=True)
final_data

,TIME_PERIOD,Country,gdp_growth_MoM
5484,1947-04,United States,0.097629
5485,1947-05,United States,0.097629
5486,1947-06,United States,0.097629
5487,1947-07,United States,0.073573
5488,1947-08,United States,0.073573
...,...,...,...
2018,2024-03,France,0.049667
2786,2024-03,Germany,0.066658
3887,2024-03,Italy,0.103363
335,2024-03,Brazil,0.211203


In [35]:
g7_countries = ["Canada", "France", "Germany", "Italy", "Japan", "United Kingdom", "United States"]

# Filter the dataframe
final_data = final_data[final_data["Country"].isin(g7_countries)]

In [36]:
final_data.to_csv("./data/gdp/gdp.csv")

In [37]:
final_data['Country'].value_counts()

Country
United States     924
United Kingdom    828
Italy             768
France            768
Japan             768
Germany           768
Canada            756
Name: count, dtype: int64